## Das t-Intervall-Verfahren für einen Mittelwert
----------------------------------------

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import t

%run ../src/notebook_env.py


---------------------------------
Working on the host: LAPTOP-9LETB4SJ

---------------------------------
Python version: 3.10.2 | packaged by conda-forge | (main, Mar  8 2022, 15:52:24) [MSC v.1929 64 bit (AMD64)]

---------------------------------
Python interpreter: C:\Users\zak\anaconda3\envs\srh\python.exe


Wir üben das Verfahren "The One-Mean t-Interval Procedure", um Konfidenzintervalle mit einem Datensatz zu konstruieren. Dazu laden wir den `students` Datensatz. Sie können die Datei `students.csv` <a href="https://userpage.fu-berlin.de/soga/200/2010_data_sets/students.csv">hier</a> herunterladen. Zunächst importieren wir den Datensatz und geben ihm einen passenden Namen.

In [2]:
# Lese Datei students.csv als Dataframe ein; Indexspalte wird übersprungen
students = pd.read_csv('students.csv', index_col=0)

Der `students` Datensatz besteht aus 8239 Zeilen, von denen jede einen bestimmten Studenten repräsentiert, und 16 Spalten, von denen jede einer Variable/einem Merkmal entspricht, das sich auf diesen bestimmten Studenten bezieht. Diese selbsterklärenden Variablen sind: *stud.id, name, gender, age, height, weight, religion, nc.score, semester, major, minor, score1, score2, online.tutorial, graduated, salary*.

In diesem Abschnitt konzentrieren wir uns wieder auf die Körpergröße von Studentinnen. Gehen wir davon aus, dass die im Datensatz angegebenen Größenmessungen eine sehr gute Annäherung an die interessierende Population darstellen, so dass wir eine Stichprobe aus der Wahrscheinlichkeitsdichtefunktion auf der Grundlage des Mittelwerts und der Standardabweichung des Datensatzes ziehen.

In [3]:
# Lese Spalte 'height' ein
genderheight = students[{'gender','height'}]
female = genderheight.loc[genderheight['gender'] == 'Female']
female = female['height']
f_mean = np.mean(female)
f_std = np.std(female)

C:\Users\zak\AppData\Local\Temp\ipykernel_11668\3478013545.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  genderheight = students[{'gender','height'}]


In [4]:
pop_mean = f_mean
pop_mean

163.65328467153284

In [5]:
f_std

7.918762263149209

Wir ziehen eine Zufallsstichprobe mit einem Stichprobenumfang von $n=6$ aus der Wahrscheinlichkeitsverteilung, die durch den Mittelwert und die Standardabweichung der Höhenvariablen definiert ist, und berechnen den Stichprobenmittelwert $\bar{x}$und die Stichprobenstandardabweichung $s$.

In [47]:
n = sample_size = 6
sample = t.rvs(n-1,f_mean,f_std,sample_size,random_state = 1)
sample_mean = np.mean(sample)
sample_mean

161.44236164394297

In [48]:
s = np.std(sample)
s

10.09685230523463

Unsere Zufallsstichprobe ergibt einen Stichprobenmittelwert $\bar{x}$ von etwa $161,44$ und eine Stichprobenstandardabweichung $s$ von ungefähr $10,1$. Dies sind unsere Punktschätzungen für die interessierende Grundgesamtheit, die in diesem Fall die Körpergröße der Studentinnen in unserem Datensatz ist.

Wie genau ist unsere Punktschätzung? Wir fragen Python, ob unsere Schätzungen mit dem wahren Grundgesamtheitsparameter übereinstimmen.

In [49]:
sample_mean == f_mean

False

In [50]:
s == f_std

False

In [51]:
s < f_std

False

OK, das haben wir erwartet!

Berechnen wir nun die Intervallschätzungen, indem wir die $90\%-$, $95\%-$ und $99\%-$Konfidenzintervalle konstruieren. Erinnern Sie sich an die Gleichung für ein Konfidenzintervall und wie man den Freiheitsgrad berechnet.

$$CI: \text{Punktschätzung} \pm t^*_{df,\,\alpha/2} \times \frac{s}{\sqrt{n}}$$

$$df = n-1$$

Der kritische Wert $t^*_{5,\,\alpha/2}$ beträgt $2,02$, $2,57$ und $4,03$ für Konfidenzniveaus von $90\%$, $95\%$ bzw. $99\%$.

Angewandt auf unsere Höhendaten ergibt die Gleichung

$$CI_{90\%}: 161,44 \pm 2,02 \times \frac{10,1}{\sqrt{6}} = 161,44 \pm 8,32$$

Wir können also mit $90 \%$iger Sicherheit sagen, dass die durchschnittliche Körpergröße der Studentinnen (der Populationsparameter $\mu$) zwischen $153,12$ und $169,76$ cm liegt.

$$CI_{95\%}: 161,44 \pm 2,57 \times \frac{10,1}{\sqrt{6}} = 161,44 \pm 10,6$$

Wir können also mit $95\%$iger Sicherheit sagen, dass die durchschnittliche Körpergröße der Schülerinnen zwischen $150,84$ und $172,04$ cm liegt.

$$CI_{99\%}: 161,44 \pm 4,03 \times \frac{10,1}{\sqrt{6}} = 161,44 \pm 16,6$$

Wir können also mit $99\%$iger Sicherheit sagen, dass die durchschnittliche Körpergröße der Studentinnen zwischen $144,84$ und $178,04$ cm liegt.

Es liegt auf der Hand, **dass die Fehlerspanne größer ist, wenn wir eine höhere Sicherheit haben wollen, dass der unbekannte Grundgesamtheitsparameter im Intervall enthalten ist**.

Zur Überprüfung der Korrektheit untersuchen wir, ob wir mit unserer Intervallschätzung tatsächlich den wahren Grundgesamtheitswert erfasst haben. Es ist wichtig, sich daran zu erinnern, dass das Konfidenzintervall unserem Stichprobenmittelwert keine Wahrscheinlichkeit zuweist, sondern besagt, dass das Konfidenzintervall bei wiederholten Zufallsstichproben den Mittelwert der Grundgesamtheit in $100(1-\alpha)\%$ der Fälle einschließen sollte. Um diese Behauptung zu testen, ändern wir die von uns im vorherigen Abschnitt geschriebene Python-Funktion leicht ab.

In [52]:
def CI_eval_t(pop_mean, sigma, n, estimate, alpha):
    out=[]
    for i in range(0,len(alpha)):
        out.append(pop_mean >= estimate - t.ppf(1-alpha[i]/2,df = n-1)*(sigma/np.sqrt(n)) and pop_mean <= estimate + t.ppf(1-alpha[i]/2,df = n-1)*(sigma/np.sqrt(n)))
    return out

Wenden wir nun unsere selbst erstellte Funktion `CI.eval.t()` auf unsere Daten an. Wir setzen `pop.mean = f_mean`, `sigma = s`, `n = sample.size`, `estimate = x.bar`, `alpha = [0.1,0.05,0.01]`, um zu evaluieren, ob die drei oben konstruierten Konfidenzintervalle ($90\%$, $95\%$ und $99\%$) den Populationsmittelwert enthalten. Schließlich wandeln wir den resultierenden Vektor in ein `dataframe` -Objekt um, um die Lesbarkeit zu verbessern.

In [57]:
n=sample_size
# Wende Funktion CI_eval_t an und speichere Ergebnis in eval
eval = CI_eval_t(pop_mean = f_mean, sigma = s, n = n, estimate = sample_mean, alpha = [0.1, 0.05, 0.01])

# Erzeuge Dataframe
df = pd.DataFrame({'eval':eval},
                  index=['90%', '95%', '99%'])
df

,eval
90%,True
95%,True
99%,True


In [61]:
sample_mean

163.5781702515895

Nun, interessantes Ergebnis. Der wahre Mittelwert der Population (`f_mean` = $163,65$) wird von allen drei Konfidenzintervallen erfasst. Der Mittelwert unserer Zufallsstichprobe (`sample_mean` = $163,58$) war ein recht guter Schätzer.

Bislang haben wir uns auf eine Stichprobe konzentriert. Was ist jedoch zu erwarten, wenn wir den Stichprobenprozess immer wieder wiederholen?

Ein Konfidenzintervall mit einem bestimmten Konfidenzniveau besagt, dass das Konfidenzintervall bei wiederholten Stichproben den wahren Populationsparameter in $100(1-\alpha)\%$

der Zeit einschließt. Ein Konfidenzintervall von $99 \%$ bedeutet also, dass die Fehlermarge im Durchschnitt in $99$ von $100$ Fällen groß genug ist, um den wahren Wert einzuschließen; dies bedeutet jedoch auch, dass wir uns im Durchschnitt in einem von $100$ Fällen irren. Bei einem Konfidenzintervall von $95 \%$ bzw. $90 \%$ liegen wir in $5$ bzw. $10$ von $100$ Fällen falsch. Wir verwenden Python, um diese Behauptung zu testen, indem wir ein Simulationsexperiment durchführen.

Wir ziehen eine Stichprobe aus der Höhenverteilung von oben. Wir nehmen $10$, $50$, $100$, $1000$ und $10000$ Zufallsstichproben. Jede Stichprobe hat einen Stichprobenumfang von $n=6$
. Für jede Stichprobe berechnen wir den Stichprobenmittelwert, der für uns als Stichprobenstatistik von Interesse ist. Durch Anwendung unserer selbst erstellten Funktion `CI_eval_t()` testen wir, ob der wahre Mittelwert der Grundgesamtheit durch das $90\%-$, $95\%-$ und $99\%-$ Konfidenzintervall erfasst wird. Wir speichern den sich daraus ergebenden booleschen Vektor und berechnen den Anteil der Fälle, in denen jedes bestimmte Konfidenzintervall tatsächlich den wahren Mittelwert der Grundgesamtheit enthält. Schließlich speichern wir den Anteil als Prozentsatz in einem `dataframe` -Objekt und benennen seine Zeilen und Spalten zur besseren Lesbarkeit um.

In [58]:
trials = [10,100,1000,10000]

for trial in trials:
    evals = []
    df = pd.DataFrame()
    # Wende Funktion CI_eval an und speichere Ergebnis in eval
    for i in range(trial):
        sample = t.rvs(trial-1,f_mean,f_std,trial)
        sample_mean = np.mean(sample)
        evals.append(CI_eval_t(pop_mean = f_mean, sigma = f_std, n = trial, estimate = sample_mean, alpha = [0.1, 0.05, 0.01]))
    df = pd.DataFrame(evals, columns = ['90', '95','99'])
    sum_df = df.sum()
    print(sum_df/trial, trial, ':Stichproben')

90    0.9
95    1.0
99    1.0
dtype: float64 10 :Stichproben
90    0.96
95    0.96
99    1.00
dtype: float64 100 :Stichproben
90    0.897
95    0.947
99    0.992
dtype: float64 1000 :Stichproben
90    0.9029
95    0.9508
99    0.9892
dtype: float64 10000 :Stichproben


Es zeigt sich, dass mit zunehmender Anzahl von Stichproben die Zahlen konvergieren und die Wahrscheinlichkeit widerspiegeln, die durch das Vertrauensniveau gegeben ist.